---

## Data Analysis

- This file differs from [2_data_analysis_1_base_data.ipynb](2_data_analysis_1_base_data.ipynb) in that it:
    - scales the base cleaned data created in [1_data_cleaning.ipynb](1_data_cleaning.ipynb).

Source dataset: 247076 rows × 37 columns
Processed and analyzed dataset: 247076 rows × 37 columns

---

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys
import ml_clean_feature as mlclean

In [2]:
start_time = time.time()

---

## 1. Read the cleaned dataset from file

---

In [3]:
# reload any changes to Config Settings
importlib.reload(config)

# BE SURE TO UPDATE THE LABEL FOR THIS ANALYSIS
# #############################
dataset_label = '2.0 StandardScaler Dataset'
# #############################

year                        = config.year

clean_file                  = config.clean_file
performance_report          = config.performance_report

report_path                 = config.report_path
file_label                  = dataset_label.lower().replace(' ','_')
detailed_performance_report = report_path + file_label + '_detailed_performance_report.txt'

print(f"Year:                        {year}")
print(f"Clean File:                  {clean_file}")
print(f"Performance Report:          {performance_report}")
print(f"Detailed Performance Report: {detailed_performance_report}")

Year:                        2015
Clean File:                  data/brfss_2015_clean.parquet.gzip
Performance Report:          reports/performance_report.pkl
Detailed Performance Report: reports/2.0_standardscaler_dataset_detailed_performance_report.txt


In [4]:
# Read final cleaned dataset from parquet file
df = pd.read_parquet(clean_file, engine="fastparquet")

In [5]:
diabetes_labels = df.columns

In [6]:
df.shape

(253680, 22)

---

## 2. Prepare the dataset for analysis

- Split the dataset into features and labels.
- Split the dataset into training and testing sets.
- Scale the dataset

---

In [7]:
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

target = 'diabetes'
# Dictionary defining modification to be made to the base dataset
operation_dict = {  'target_column'     :  target,
                    'convert_to_binary' :  False,
                    'scaler'            : 'standard', # options: none, standard, minmax
                    'random_sample'     : 'none'      # options: none, undersample, oversample
                    }

# This insures that df if not modified during the call to modify_base_dataset()
df_modified = df.copy()

# Modify the base dataset
# data is returned where: X_train, X_test, y_train, y_test = data
data = mlanlys.modify_base_dataset(df_modified, operation_dict)

Base Dataset Modifications in Process
-------------------------------------
**Operation:target_column  diabetes
**Operation:convert_to_binary  False
**Operation:scaler  standard
  -- Performing train_test_split on dataframe with target:'diabetes'
     -- Run automatically before scalar or random_sample operations
  -- Performing StandardScaler on X_train: Updates X_train, y_test
**Operation:random_sample  none

Dataframe, Train Test Summary
-----------------------------
Dataframe: (253680, 22)  Data:4, X_train:190260, y_train:190260, X_test:63420, y_test:63420
ValueCounts:   y_train: len:2   0: 160385   1:  3460
ValueCounts:   y_test : len:2   0:  53318   1:  1171


In [9]:
# Print some statistics about the original df and the modified dataframe
print(f"Original Dataframe")
print(f"------------------")
print(f"df.shape: {df.shape}")
print(f"df[{target}].value_counts:  {df[target].value_counts()}")

print(f"\nModified Dataframe")
print(f"------------------")
print(f"df_modified.shape: {df_modified.shape}")
print(f"df_modified[{target}].value_counts:  {df_modified[target].value_counts()}")

Original Dataframe
------------------
df.shape: (253680, 22)
df[diabetes].value_counts:  diabetes
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

Modified Dataframe
------------------
df_modified.shape: (253680, 22)
df_modified[diabetes].value_counts:  diabetes
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64


In [10]:
X_train, X_test, y_train, y_test = data
print(f"Dataframe: {df_modified.shape}  Data:{len(data)}, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")
y_train.value_counts()

Dataframe: (253680, 22)  Data:4, X_train:190260, y_train:190260, X_test:63420, y_test:63420


diabetes
0.0    160385
2.0     26415
1.0      3460
Name: count, dtype: int64

In [11]:
y_test.value_counts()

diabetes
0.0    53318
2.0     8931
1.0     1171
Name: count, dtype: int64

---

## 3. Run initial Tests and get k_value

**From step 2:**  Data = [X_train_modified, X_test_modified, y_train_modified, y_test]

---

In [12]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

# Determine the k_value
# mlanlys.knn_plot(data)

<module 'ml_analysis' from '/mnt/c/ML/DU/repos/projects/project-2/DU-project-2-2015/brfss_2015/../pkgs/ml_analysis.py'>

**Note:** From the knn plot above, pick a k-value of 3.

---

## 4. Run the Analysis

---

#### Model Run Times

-  Base dataset (247076 rows × 37 columns):

| Model | Run Time |
| ----- | -------- |
| test_model(SVC(kernel='linear'), data)                          | Aborted >35min (Data too large, consider for RandomUndersampling dataset) |
| test_model(KNeighborsClassifier(n_neighbors=k_value), data)     | 247.13 seconds |
| test_model(tree.DecisionTreeClassifier(), data)                 |   3.89 seconds |
| test_model(RandomForestClassifier(), data)                      |  60.94 seconds |
| test_model(ExtraTreesClassifier(random_state=1), data)          |  58.54 seconds |
| test_model(GradientBoostingClassifier(random_state=1), data)    | 115.21 seconds |
| test_model(AdaBoostClassifier(random_state=1), data)            |  11.91 seconds |
| test_model(LogisticRegression(), data)                          |   4.90 seconds |
| **Total** w/o SVC| 502.52 seconds / **8:23 minutes** |

In [13]:
# reload any changes to nlanlys
importlib.reload(mlanlys)

k_value = 3

#### COMMENT OUT ONE OF THE FOLLOWING SECTIONS

## SECTION 1
# Capture stdout & stderr into two strings: osc.stdout and osc.stderr that contain the output from the function
# -- This allows the output to be printed here or to a file or both.

with mlanlys.OutStreamCapture() as osc:
    performance_summary = mlanlys.run_classification_models(data, k_value)
#    performance_summary = mlanlys.run_classification_models_test(data, k_value)

## <OR>
## SECTION 2

# performance_summary = mlanlys.run_classification_models(data, k_value)


In [14]:
# UNCOMMENT if using SECTION 1 in the previous step
# print(osc.stdout)

# Add code to print osc.stdout to a file if desired.

---

### 4.1 Archive Performance Summary

- For use in Project-2 Performance Summary Report
---

In [15]:
# performance_summary is a dataframe of performance statistics

# Add the dataset label as the first column in performance_summary
dataset_column = pd.Series([dataset_label] * len(performance_summary), name=dataset_label)
performance_summary.insert(0, 'new_column', dataset_column)

analysis_perf_summary = { 'dataset_size': list(df.shape), 'report': performance_summary}

# Performance_report is a file containing all the performance summary statistics
if os.path.exists(performance_report):
    print(f"The file {performance_report} exists.")
    # Load Performance Report
    with open(performance_report, 'rb') as file: perf_report = pickle.load(file)
else:
    print(f"The file {performance_report} does not exist.")
    perf_report = {}
    
perf_report[dataset_label] = analysis_perf_summary

# Save Performance Report
with open(performance_report, 'wb') as file: pickle.dump(perf_report, file)

The file reports/performance_report.pkl does not exist.


### 4.2 Archive the Performance Detailed Statistics Report
---

In [16]:
# osc.stdout contains the details of the performance statistics

with open(detailed_performance_report, "w") as file:
    file.write(osc.stdout)

---

## 5. Performance Summary

---

In [17]:
# print the performance summary
print(f"******************************************")
print(f"Performance Summary for: {dataset_label}")
print(f"******************************************")

performance_summary

******************************************
Performance Summary for: 2.0 StandardScaler Dataset
******************************************


,new_column,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,2.0 StandardScaler Dataset,KNeighborsClassifier,Train,0.8866,0.5038,0.9481,0.4014,0.9027,0.7260,0.4801,0.5780,0.9708,0.0292,NaN
1,2.0 StandardScaler Dataset,KNeighborsClassifier,Test,0.8194,0.3917,0.6253,0.6637,0.8355,0.3696,0.2378,0.2894,0.9335,0.0665,0.2076
2,2.0 StandardScaler Dataset,DecisionTreeClassifier,Train,0.9934,0.9684,0.9999,0.0230,0.9944,0.9993,0.9606,0.9796,0.9999,0.0001,NaN
3,2.0 StandardScaler Dataset,DecisionTreeClassifier,Test,0.7651,0.4030,0.5675,0.8117,0.7977,0.2968,0.3190,0.3075,0.8761,0.1239,0.1893
4,2.0 StandardScaler Dataset,RandomForestClassifier,Train,0.9934,0.9689,0.9995,0.0231,0.9944,0.9944,0.9651,0.9795,0.9991,0.0009,NaN
5,2.0 StandardScaler Dataset,RandomForestClassifier,Test,0.8414,0.3884,0.7386,0.5762,0.8589,0.4972,0.1974,0.2826,0.9673,0.0327,0.2493
6,2.0 StandardScaler Dataset,ExtraTreesClassifier,Train,0.9934,0.9684,0.9999,0.0230,0.9944,0.9993,0.9606,0.9796,0.9999,0.0001,NaN
7,2.0 StandardScaler Dataset,ExtraTreesClassifier,Test,0.8330,0.3862,0.7170,0.6057,0.8513,0.4380,0.1966,0.2714,0.9587,0.0413,0.2219
8,2.0 StandardScaler Dataset,GradientBoostingClassifier,Train,0.8508,0.3913,0.7957,0.5424,0.8672,0.5623,0.1955,0.2901,0.9755,0.0245,0.2758
9,2.0 StandardScaler Dataset,GradientBoostingClassifier,Test,0.8490,0.3903,0.7885,0.5485,0.8659,0.5702,0.1934,0.2888,0.9761,0.0239,0.2764


In [18]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

Completed: Execution Time 373.76 seconds:


---

## 6. Conclusions

- A first glance at the summary, it appears that the Boosting models may have performed well with test/train scores were >.8 and similar in scale (<.02 delta).  However, the poor test confusion matrix and balanced accuracy highlight the overfitting.

- The Base Cleaned data is overfit as indicated by:
    - Poor confusion matrix on the detailed report for test sets on all models
    - Low balanced accuracy as compared to the model score (less than 50%)


---

---